In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import time
import re
import nltk
import math
import os
import matplotlib.pyplot as plt
import sklearn.metrics
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score

%matplotlib inline


In [2]:
def fasta_parser(filename):
    if os.path.exists(filename):
        pass
    else:                     
        print("The file, %s, does not exist" % filename)
        
    file = open(filename,mode='r')
    rec_all = file.read()
    file.close()
    # remove all whitespace from string all_of_it
    rec_all = rec_all.replace(' ','')
    # split records by > 
    records = rec_all.split('>')
    # Parse out the headers & sequences for each record
    headers = []
    sequences = []
    for rec in records:
        s = ''
        data = rec.split('\n')
        sq = s.join(data[1:])
        if len(data[0]) > 0:
            headers.append(data[0])
   
        if (len(sq) > 0):
            sequences.append(s.join(data[1:]))
    #print(sequences)
    return headers, sequences

## The dataset contains 5959 proteins annotated to one of 11 different subcellular locations which are: chloroplast, cytoplasm, endoplasmic reticulum, extracellular space, Golgi apparatus, lysosomal, mitochondrion, nucleus, peroxisome, plasma membrane and vacuole which represented proteins of plants cell and fungal cell while animal cells shared all localizations with them, but have lysosomes instead of vacuoles. The only variable we intend to consider is protein sequence.


In [3]:
# data downloaded from kaggle
# https://www.kaggle.com/lzyacht/proteinsubcellularlocalization/downloads/proteinsubcellularlocalization.zip/1
# data is from SWISS-PROT database release 42 (2003–2004)
df = pd.read_csv('proteinsLocations.csv')
df.head(3)

,label,name,sequence
0,chloroplast.fasta,P46644 cTP 43,MKTTHSSSSSSDRRGARHNSGSDSDNSSYASTSGGTGGSVSHVADG...
1,chloroplast.fasta,P46248 cTP 52,MASMSGSTSRNKDKKGTSASNKAKSSRVTMTVAVKSRGTMADGVSA...
2,chloroplast.fasta,Q96375 cTP 49,MYASSARDGGKWCNARRKSKDAYHSCKSNGHKKVKGVKATAAATTK...


In [4]:
df.shape

(5959, 3)

In [5]:
df.dtypes

label       object
name        object
sequence    object
dtype: object

In [6]:
df.label.value_counts()

cytoplasmic.fasta        1411
plasma_membrane.fasta    1238
extracellular.fasta       843
nuclear.fasta             837
mitochondrial.fasta       510
chloroplast.fasta         449
ER.fasta                  198
peroxisomal.fasta         157
Golgi.fasta               150
lysosomal.fasta           103
vacuolar.fasta             63
Name: label, dtype: int64

In [7]:
df.isnull().sum()

label       0
name        0
sequence    0
dtype: int64

## Encode different subcellular locations as integers

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.label)
df['location'] = le.transform(df['label'])

In [9]:
df.head(3)

,label,name,sequence,location
0,chloroplast.fasta,P46644 cTP 43,MKTTHSSSSSSDRRGARHNSGSDSDNSSYASTSGGTGGSVSHVADG...,2
1,chloroplast.fasta,P46248 cTP 52,MASMSGSTSRNKDKKGTSASNKAKSSRVTMTVAVKSRGTMADGVSA...,2
2,chloroplast.fasta,Q96375 cTP 49,MYASSARDGGKWCNARRKSKDAYHSCKSNGHKKVKGVKATAAATTK...,2


In [10]:
df.tail()

,label,name,sequence,location
5954,vacuolar.fasta,Q9P7E9 SA 67 89,MNAYGDTNNHGKSSTRHWRKRSAVSSSSSYSNSNTVKVSAKKRRRK...,10
5955,vacuolar.fasta,P33894 SA 120 140,MSASTHSHKRKNSHRKSSNSSMDKNNDSVANTDSNNGHTNRTATDV...,10
5956,vacuolar.fasta,P18962 SA 30 45,MGGVRDDTKKKHDKRVGVWGTVKSHHSNTDYNSNYTNDGKKVSSVV...,10
5957,vacuolar.fasta,P49047 SP 20,MTTVVSAVAAVSGDVKSASKRTNDDDSTKWAVVAGSSGYWNYRHAD...,10
5958,vacuolar.fasta,Q39044 SP 21,MAKSCYRAVVHASRGRKMTANADDDGVGTRWAVVAGSSGYGNYRHA...,10


In [11]:
df.dtypes

label       object
name        object
sequence    object
location     int64
dtype: object

In [12]:
df.location.value_counts()

3     1411
9     1238
4      843
7      837
6      510
2      449
0      198
8      157
1      150
5      103
10      63
Name: location, dtype: int64

## Select only data for proteins in the cytoplasm or plasma membrane for binary classification

In [13]:
# select only two types of proteins: cytoplasmic and plasma membrane proteins
mask = ((df.location == 3) | (df.location == 9))
#mask = ((df.location == 3) | (df.location == 6))
data = df[mask]
data.head()

,label,name,sequence,location
449,cytoplasmic.fasta,P31946,TMDKSVKAKAARYDDMAAAMKAVTGHSNRNSVAYKNVVGARRSSWR...,3
450,cytoplasmic.fasta,P42655,MDDRDVYAKAARYDMVSMKKVAGMDVTVRNSVAYKNVGARRASWRS...,3
451,cytoplasmic.fasta,P35214,VDRVKARAARYDDMAAAMKNVTNSNRNSVAYKNVVGARRSSWRVSS...,3
452,cytoplasmic.fasta,Q91896,MDKNVKAKAARYDDMAACMKRVTGGSNRNSVAYKNVVGARRSSWRV...,3
453,cytoplasmic.fasta,Q15172,MSSSSAGAASAASASKVDGTRKSVRKARKRSGSSRSGSAHKDATSN...,3


In [14]:
# scramble the data
data = data.sample(frac=1)
data.head(3)

,label,name,sequence,location
5353,plasma_membrane.fasta,O75473 SP 21,MDTSRGVSVATGGSSRSGVRGCTHCHCDGRMRVDCSDGSSNSVTSY...,9
874,cytoplasmic.fasta,P48597,MAAVNTNSTKTGVSDYKHNRWAWKNDKSKTWANRSKDTVDWAYNHS...,3
1494,cytoplasmic.fasta,O13426,MSAYASSHRTGHKDTDVDKDDRHSVASNTTTAVDAGTMCNKYSGYG...,3


In [15]:
data.location.value_counts()

3    1411
9    1238
Name: location, dtype: int64

## Transform the data with CountVectorizer

In [16]:
vect_3 = CountVectorizer(min_df=1,token_pattern=r'\w{1}',ngram_range=(3, 3))

X = vect_3.fit_transform(data.sequence)
y = data.location

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state =42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2119, 2763) (2119,)
(530, 2763) (530,)


In [17]:
print(len(y_train))
print(len(y_test))

2119
530


In [18]:
y_train.value_counts()

3    1112
9    1007
Name: location, dtype: int64

In [19]:
y_test.value_counts()

3    299
9    231
Name: location, dtype: int64

## Logistic Regression 

In [20]:
# Logistic Regression using CountVectorizer for tripeptide frequency
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
print("Score: {:.2f}".format(lr.score(X_test, y_test)))

Score: 0.91


In [21]:
# Try optimizing Logistic Regression model
#the grid of parameters to search over
Cs = [0.001,0.01, 0.1, 1, 10, 100]

Scores = []

for item in Cs:
    clf = LogisticRegression(C=item)
    clf.fit(X_train, y_train)
    Scores.append((clf.score(X_test, y_test)))
    
Scores

[0.91509433962264153,
 0.91509433962264153,
 0.91320754716981134,
 0.91320754716981134,
 0.90943396226415096,
 0.90943396226415096]

In [22]:
score_highest = max(Scores)
print(score_highest)
print()
print(Scores.index(score_highest))
print()
C_opt = Cs[Scores.index(score_highest)]
print(C_opt)
print()

0.915094339623

0

0.001



In [23]:
# Logistic Regression using CountVectorizer for tripeptide frequency Optimized
lr2 = LogisticRegression(C=C_opt)
lr2.fit(X_train, y_train)
#predictions = lr2.predict(X_test)
print("Score: {:.2f}".format(lr2.score(X_test, y_test)))

Score: 0.92


## Now try the same analysis: tripeptide frequency & Logistic Regression on ALL the data for multiclass classification

In [24]:
vect_3 = CountVectorizer(min_df=1,token_pattern=r'\w{1}',ngram_range=(3, 3))
#vect_3 = CountVectorizer(min_df=1,token_pattern=r'\w{1}',ngram_range=(5, 5))
X = vect_3.fit_transform(df.sequence)
y = df.location

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state =42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4767, 2764) (4767,)
(1192, 2764) (1192,)


In [25]:
print(len(y_train))
print(len(y_test))

4767
1192


In [26]:
y_train.value_counts()

3     1117
9      996
7      676
4      670
6      424
2      343
0      165
1      125
8      117
5       87
10      47
Name: location, dtype: int64

In [27]:
y_test.value_counts()

3     294
9     242
4     173
7     161
2     106
6      86
8      40
0      33
1      25
10     16
5      16
Name: location, dtype: int64

In [28]:
# Logistic Regression using CountVectorizer for tripeptide frequency
lr_all = LogisticRegression()
lr_all.fit(X_train, y_train)
predictions = lr_all.predict(X_test)
print("Score: {:.2f}".format(lr_all.score(X_test, y_test)))

Score: 0.57


In [29]:
# Try optimizing Logistic Regression model
#the grid of parameters to search over
Cs = [0.001,0.01, 0.1, 1, 10, 100]

Scores = []

for item in Cs:
    clf = LogisticRegression(C=item)
    clf.fit(X_train, y_train)
    Scores.append((clf.score(X_test, y_test)))
    
Scores

[0.50335570469798663,
 0.59983221476510062,
 0.60151006711409394,
 0.5738255033557047,
 0.56291946308724827,
 0.55620805369127513]

In [30]:
score_highest = max(Scores)
print(score_highest)
print()
print(Scores.index(score_highest))
print()
C_opt = Cs[Scores.index(score_highest)]
print(C_opt)
print()

0.601510067114

2

0.1



In [31]:
# Logistic Regression using CountVectorizer for tripeptide frequency Optimized
lr2 = LogisticRegression(C=C_opt)
lr2.fit(X_train, y_train)
#predictions = lr2.predict(X_test)
print("Score: {:.2f}".format(lr2.score(X_test, y_test)))

Score: 0.60


## Try Again With Logistic Regression

In [32]:
lbf = LogisticRegression(solver = 'lbfgs',multi_class='ovr').fit(X_train, y_train)
#print the test accuracy score
print("Score: {:.2f}".format(lbf.score(X_test, y_test)))

Score: 0.58


## Yikes!, the Logistic Regression model did poorly trying to classify all 11 cellular locations using this approach. In contrast, Logistic Regression with tripeptide frequency worked well classifying plasma membrane proteins from cytoplasmic proteins. 

## KNN Classification

In [33]:
knn = KNeighborsClassifier(n_neighbors = 3).fit(X_train, y_train) 

# print knn score
print("Score: {:.2f}".format(knn.score(X_test, y_test)))

Score: 0.33


## Multiclass Label Classification: One vs. Rest

In [34]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

ovrClf = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
ovrClf.predict(X_test)

array([9, 3, 4, ..., 3, 3, 3])

In [35]:
# print the score
print("Score: {:.2f}".format(ovrClf.score(X_test, y_test)))

Score: 0.53


## Gradient Boosting Classifier

In [36]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train,y_train)
print("Score: {:.2f}".format(gbc.score(X_test, y_test)))

Score: 0.56


## Decision Tree Classifier

In [37]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
predictions = dtc.predict(X_test)
print("Score: {:.2f}".format(dtc.score(X_test, y_test)))

Score: 0.30
